In [1]:
import gym
import random, math
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 10)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
torch.manual_seed(0)

import base64, io

# For visualization
from gym.wrappers.monitoring import video_recorder
from IPython.display import HTML
from IPython import display
import glob
from reinforce_PPORLHF import reinforce_rwd2go_PPO_RLHF
from utils import pref_save, pref_load

from model import Policy
import pickle

%load_ext autoreload
%autoreload 2
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [2]:
K = 1000
SEED=0
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

env = gym.make('CartPole-v0')
print('observation space:', env.observation_space)
print('action space:', env.action_space)

observation space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
action space: Discrete(2)


/Users/guillaumevitalis/miniconda3/envs/RLA2Final/lib/python3.9/site-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/Users/guillaumevitalis/miniconda3/envs/RLA2Final/lib/python3.9/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/Users/guillaumevitalis/miniconda3/envs/RLA2Final/lib/python3.9/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
policy1 = Policy().to(device)
policy1.load_state_dict(torch.load(f"policy1_{K}.pth"))
policy2 = Policy().to(device)
policy2.load_state_dict(torch.load(f"policy2_{K}.pth"))

pref_data = pref_load(f"pref_data_{K}.pickle")

In [4]:
class RewardModel(nn.Module):
    def __init__(self, state_size=4, action_size=1, hidden_size=32):
        super(RewardModel, self).__init__()
        self.fc1 = nn.Linear(state_size + action_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = self.fc2(x)
        # we just consider 1 dimensional probability of action
        return x

    def predict_reward(self, state, action):
        state = state
        action = torch.tensor(action).reshape(1,1)  
        
        state_action = torch.cat((state, action), dim=1)
        reward = self.forward(state_action).cpu()
        return reward

In [5]:
import torch
import torch.nn.functional as F
import copy

lr        = 3e-3
epochs    = 10

reward_model = RewardModel(state_size=4, action_size=1)

optimizer = torch.optim.Adam(reward_model.parameters(), lr=lr)

def trajectory_reward(reward_model, states, actions):
    total_reward = torch.tensor(0., device=device)
    for s, a in zip(states, actions):
        s_t = torch.tensor(s, dtype=torch.float32, device=device)
        reward = reward_model.predict_reward(s_t.unsqueeze(0), a).squeeze(0)
        total_reward += reward.squeeze(0)
    return total_reward

for epoch in range(1, epochs+1):
    total_loss = 0.0
    
    for s0, tau_plus, tau_minus in pref_data:

        reward_plus = trajectory_reward(reward_model, tau_plus["states"], tau_plus["actions"])
        reward_minus = trajectory_reward(reward_model, tau_minus["states"], tau_minus["actions"])
        total_loss += - torch.log(torch.exp(reward_plus) / (torch.exp(reward_minus) + torch.exp(reward_plus)))

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    avg_loss = total_loss / len(pref_data)
    
    print(f"Epoch {epoch}/{epochs} — avg loss: {avg_loss:.4f}")

Epoch 1/10 — avg loss: 2.2837
Epoch 2/10 — avg loss: 1.1696
Epoch 3/10 — avg loss: 0.4279
Epoch 4/10 — avg loss: 0.0989
Epoch 5/10 — avg loss: 0.0177
Epoch 6/10 — avg loss: 0.0032
Epoch 7/10 — avg loss: 0.0005
Epoch 8/10 — avg loss: 0.0001
Epoch 9/10 — avg loss: 0.0000
Epoch 10/10 — avg loss: 0.0000


In [8]:
policy2 = Policy().to(device)
policy2.load_state_dict(torch.load(f"policy2_{K}.pth"))
opt1    = optim.Adam(policy2.parameters(), lr=1e-3)
reward_model.eval()
reinforce_rwd2go_PPO_RLHF(env, policy2, opt1, reward_model, n_episodes=2000);

Ep 100	avg100: 74.59
Ep 200	avg100: 57.95
Ep 300	avg100: 60.48
Ep 400	avg100: 59.20
Ep 500	avg100: 70.27
Ep 600	avg100: 109.54
Ep 700	avg100: 109.53
Ep 800	avg100: 120.39
Ep 900	avg100: 131.27
Ep 1000	avg100: 153.30
Ep 1100	avg100: 175.48
Ep 1200	avg100: 193.39
Ep 1300	avg100: 189.43
Ep 1400	avg100: 188.45
Ep 1500	avg100: 191.20
Ep 1600	avg100: 181.73
Ep 1700	avg100: 161.46
Ep 1800	avg100: 196.53
Ep 1900	avg100: 198.64


In [7]:
returns = []
eval_episodes = 100
for ep in range(eval_episodes):
    state, done, total_r = env.reset(), False, 0.0
    while not done:
        # choose greedy or stochastic—here greedy
        with torch.no_grad():
            s_t = torch.tensor(state, dtype=torch.float32, device=device)
            probs = policy2(s_t.unsqueeze(0)).squeeze(0)
            action = torch.argmax(probs).item()
        state, r, done, _ = env.step(action)
        total_r += r
    returns.append(total_r)

mean_return = sum(returns) / len(returns)
print(f"Evaluation over {eval_episodes} episodes: mean return = {mean_return:.2f}")

Evaluation over 100 episodes: mean return = 150.02
